# PLAsTiCC Astronomical Classification Tutorial

## Introduction

### The Large Synoptic Survey Telescope (LSST)

The PLAsTiCC team is responsible for the Large Synoptic Survey Telescope (LSST), and has plans of revolutionizing the field of astronomy. This telescope will enable astronomers to discover 10 to 100 times the amount of astronomical sources in the night sky than we have ever been able to measure before.

The LSST was under construction in a mountain in Chile during the time of the Kaggle competition, but will be operational in 2019. When complete, it will be 8-meters long and contain a 3-billion pixel camera. The dataset in this competition contains simulated timeseries for different objects in the sky, based on observations one might expect to see from the real telescope.

### Light curves

The LSST measures *light curves*, which are an object's brightness as a function of time. These light curves are captured by taking measurements of each object's photon flux in six different astronomical light filters, called passbands. The passbands include the ultra-violet, optical, and infrared regions of the light spectrum. The mode used to collect measures in the LSST is referred to as *photometry*.

The light curves result from taking two images in the same region of the sky on different nights and subtracting them. This differencing procedure is able to capture both moving objects, like asteroids (which are removed), and objects that stay in the same position but vary in brightness. The flux may decrease or increase with time, depending on when the object first exploded or brightened. 

### Objective

Astronomical objects are driven by different physical processes and astronomers separate these objects into different astronomical classes. The manner in which the flux changes is a good indicator of the object's class. The dataset in this competition contains a set of training light curves (time series), which contain labels for 14 different object classes. The testing dataset is unlabeled, and will contain objects from 15 different classes. The goal is to classify the astronomical objects into these 15 different classes.


### Data

A great, and comprehensive, description of the dataset can be found [here](https://www.kaggle.com/c/PLAsTiCC-2018/data).

#### The columns in the training set contain the following fields:
- **object_id** [int32]: Unique id used to identify astronomical objects.
- **mjd** [float64]: the time in Modified Julian Date (MJD) of the observation.
- **passband** [int8]: The specific LSST passband integer, such that u, g, r, i, z, Y = 0, 1, 2, 3, 4, 5 in which it was viewed.
- **flux** [float32]: the measured flux (brightness) in the passband of observation as listed in the passband column.
- **flux_err** [float32]: the uncertainty on the measurement of the flux measurement.
- **detected** [boolean]: If 1, the object's brightness is significantly different at the 3-sigma level relative to the reference template. Only objects with at least 2 detections are included in the dataset.

#### Caveats / Additional Challenges

- There gaps in the times at which passband observations were taken, sometimes many days apart
- Galactic vs extragalactic: The given redshift for objects in our own Milky Way
galaxy is given as zero.
- Due to statistical fluctuations (of e.g. the sky brightness) and the
way the brightness is estimated, the flux may be negative for dim sources, where
the true flux is close to zero. Additionally, if the pre-survey image actually contains a
flux brighter than its true ‘zero’, this can lead to a negative flux when the difference
is computed.


This introduction was adapted from [The Photometric LSST Astronomical
Time-series Classification Challenge, The PLaSTiCC Team et al, 2018](https://arxiv.org/pdf/1810.00001.pdf). Refer to this reference for more details. 

## Solution

This tutorial will explore the [8th place](https://www.kaggle.com/c/PLAsTiCC-2018/leaderboard) solution to the challenge, which uses the RAPIDS open-source ecosystem of data science tools for GPU-accelerated feature engineering and model building. It has been simplified from the [original solution](https://devblogs.nvidia.com/make-sense-universe-rapids-ai/) in order to fit within a 40-minute tutorial session. 

The diagram below outlines the full end-to-end solution, which uses RAPIDS cuDF to engineer features for training XGBoost, MLP, and Bidirectional (attentional) RNN models. This tutorial will include the feature engineering and training processes for the XGBoost and RNN models. The training of the MLP model and the [model stacking](https://towardsdatascience.com/automate-stacking-in-python-fc3e7834772e) are left as future work for the students.

<img src="https://devblogs.nvidia.com/wp-content/uploads/2019/02/1tv0fuN-usAsQLJzTj1zSLQ.png" width=700/>

For the sake of time, we will be implementing a simplified version of the end-to-end Kaggle solution. In this simplified version, we will
1. engineer timeseries features by embedding timeseries into a 16-dimensional space using a pre-trained RNN model
2. engineer aggregated features by building statistical summaries from features in the input data, and 
3. train an XGBoost classifier using both our timeseries embeddings & our statistical features to classify the data in the test set

<hr/>

### This tutorial is broken down into two parts:

#### Part 1 - Training RNN Timeseries Embeddings
- **Part 1.1:** Engineer features to train an RNN for converting our timeseries into fixed-length vectors
- **Part 1.2:** Uses the pre-trained RNN to map timeseries to the fixed-length vectors

#### Part 2 - Training an XGBoost Classifier
- **Part 2.1:** Engineers features for each object id by aggregating features across observations
- **Part 2.2:** Combines features from Parts *1.2* and *2.1* to train and evaluate an XGBoost classifier

### Conclusion

By the end of this tutorial, the student should feel comfortable doing common data processing tasks on GPUs using cuDF, training models in XGBoost, and integrating cuDF-built datasets into popular deep learning frameworks like [Tensorflow](https://www.tensorflow.org/). 